### 1. Find all 'Posxx-xx.txt' files in the sub_folder called ROI_dir_name

In [33]:
def find_all_ROI(ROI_dir_name,fileextension='.txt'):
    """input:
             ROI_dir_name"""
    from pathlib import Path
    ROI_path=Path(ROI_dir_name)#Make a path object
    ROI=list(ROI_path.glob('Pos*%s' %(fileextension)))#WindowsPath('ROI/Pos001-001.txt'), need to str(x) when saving in windows
    return ROI

In [58]:
find_all_ROI('ROI_test',fileextension='.txt')[0:2]

[WindowsPath('ROI_test/Pos10-02.txt')]

### 2: Open the 'Posxx-xx.txt' files and extract info from filename and contents

In [59]:
import pandas as pd
pd.read_table(find_all_ROI('ROI_test',fileextension='.txt')[0]).head(2)

,,Label,Area,X,Y
0,1,Pos10CFP.tif:0001-0661-0558:1,361,558.924,660.467
1,2,Pos10CFP.tif:0002-0669-0553:2,366,553.929,668.574


In [39]:
def get_Pos_cellid(this_file):
    """get_Pos_cellid(this_file) function extracts Postion and cell_id from the filename of 'Posxx-xx.txt'"""
    import re
    input = this_file.name # e.g: Pos10-01.txt
    output = re.search(r"(?P<Position>Pos[0-9]{1,3})-(?P<cell_id>[0-9]{1,3}).txt",input)
    Pos = output.group('Position')
    cellid = output.group('cell_id') 
    #return (Pos,cellid)
    #use namedtuple to return Pos, cellid
    from collections import namedtuple
    Features = namedtuple('Features',['Pos','cellid'])
    extractedinfo = Features(Pos=Pos,cellid=cellid)
    return(extractedinfo)

In [65]:
get_Pos_cellid(find_all_ROI('ROI',fileextension='.txt')[0]).cellid

'001'

In [73]:
def get_x_y_frames(this_file):
    """get_x_y_frames (this_file) function extracts 'x_coor','y_coor' and 'Label' from the file content;
    and then extract 'frames' from Label 'Pos10CFP.tif:0001-0575-0555:1'
    output 'frames' = [1,2,3]"""
    import re
    
    #this_data=pd.read_table(this_file)
    this_data=pd.read_csv(this_file,delimiter='\t')
    Label=this_data['Label']
    x=list(this_data['X'])#converting to list with number from txt file results in sciprecision
    y=list(this_data['Y'])

    num_label = len(Label)
    frames=[] 
    for i in range(num_label):
        input = Label[i] # e.g: Pos01CFP.tif:0001-0450-0596:1
        regularexpression=r"tif.(?P<f>[0-9]{1,4})-(?P<x>[0-9]{1,4})-(?P<y>[0-9]{1,4}):(?P<frame>[0-9]{1,3})"
        output=re.search(regularexpression,input)
        frame=output.group('frame')
        frames.append(int(frame))
        
    #return(x,y,frames)  
    #use namedtuple to store x,y,frames to tuple 
    from collections import namedtuple
    Features = namedtuple('Features',['x','y','frames'])
    extractedinfo = Features(x=x,y=y,frames=frames)
    return(extractedinfo)

In [75]:
import numpy as np
def extract_all(ROI_dir_name,check_or_not):
    """extract_all(ROI_dir_name,check_or_not) function extracts all required information from both filename and filecotents 
    of all files in the format of ROI/Pos01-01.txt' """
    
    ROI = find_all_ROI(ROI_dir_name,fileextension='.txt') #e.g 'ROI/Pos01-01.txt','ROI/Pos12-01.txt''
    
    Position=[]
    cell_id=[]
    x_coor=[]
    y_coor=[]
    frames=[]
    number_files = len(ROI)
    
    for i in range(number_files): 
        this_file = ROI[i]
        #partA = get_Pos_cellid(this_file)
        Position.append(get_Pos_cellid(this_file).Pos)
        cell_id.append(get_Pos_cellid(this_file).cellid)
        x_coor.append(get_x_y_frames(this_file).x)
        y_coor.append(get_x_y_frames(this_file).y)
        frames.append(get_x_y_frames(this_file).frames)
    
    if check_or_not==1:
       return (Position,cell_id,x_coor,y_coor,frames)
    else:
       #extract Position and cell_id from the name 'Posxx-xx.txt' itself
       log_output = {'Position':Position,'cell_id':cell_id,'frames':frames,'x_coor':x_coor,'y_coor':y_coor}
       dfa = pd.DataFrame(log_output)
       return dfa

In [76]:
extract_all('ROI_test',1)

(['Pos10'],
 ['02'],
 [[558.924, 553.929, 548.5790000000001, 537.885, 518.58]],
 [[660.467, 668.574, 673.39, 677.296, 684.414]],
 [[1, 2, 3, 4, 5]])

In [60]:
extract_all('ROI_test',0).head(1)

,Position,cell_id,frames,x_coor,y_coor
0,Pos10,02,"[1, 2, 3, 4, 5]","[558.924, 553.929, 548.5790000000001, 537.885,...","[660.467, 668.574, 673.39, 677.296, 684.414]"


In [63]:
extract_all('ROI_test',0)['x_coor'][0]

[558.924, 553.929, 548.5790000000001, 537.885, 518.58]

In [9]:
extract_all('ROI',0).at[0,'cell_id']

'001'

In [10]:
type(extract_all('ROI',0).at[0,'cell_id'])

str

### Make a

In [11]:
def imageshow(movie_dir_name,this_movie_name,logs_output, interval, this_position_index,cmap_type='gray'):
    """ddd"""
    import imageio #Read the multiframe.tiff movie
    import matplotlib.pyplot as plt 
    %matplotlib inline
    from matplotlib import cm
    from pathlib import Path
    
    movie_dir=Path(movie_dir_name)#Make a path object
    vol = imageio.volread(movie_dir / this_movie_name)# define movie path 'Movie/Pos100CFP.tif'
    nslices,x_size,y_size = vol.shape #number of elements along each axis
    
    #fig=plt.figure()
    #show frame by frame
    for j in range(nslices):  
        image=vol[j]#starts from slice 0
        
        plt.imshow(image,cmap_type)
        #plt.title(this_movie_name)
        plt.axis('off')
        #Labell all cell_ids for each frame
        style=dict(size=12,color='white')
        this_time = '{0:.1f}'.format(j*interval/60) #20min/frame, in the unit of hour
        time_label = ('%shr' %(this_time))
        plt.text(200,50,time_label,ha='right',**style)

        #label all cell for this frame
        num_cells_this_position = len(this_position_index)
        cellid_color=iter(cm.rainbow(np.linspace(0,1,num_cells_this_position)))
        #
        for n in range(num_cells_this_position):
            cell_id = logs_output.at[this_position_index[n],'cell_id']
            #find the index of slice j from data frame
            frames = logs_output.at[this_position_index[n],'frames']
            if (j+1 in frames)==True:
                this_frame_index = frames.index(j+1)#
                x_coor = logs_output.at[this_position_index[n],'x_coor'][this_frame_index]
                y_coor = logs_output.at[this_position_index[n],'y_coor'][this_frame_index]
                labelcolor = next(cellid_color)
                plt.text(x_coor,y_coor,cell_id,ha='right',size=12,color = labelcolor)#all images were shown in the same fig
            else:
                continue
        plt.show()
        plt.close()
        #fig.clear()
            
#for each slice, label filtered_cells for that position
def plot_all_images(logs_output,movie_dir,channel,interval):
    #Find all positions
    unique_position = logs_output['Position'].unique().tolist() #['Pos01', 'Pos10']
    num_position = len(unique_position)
    #for each posxxCFP.tif movie
    for i in range(num_position):
        #
        this_position = unique_position[i]  
        this_movie_name = ('%s%s.tif' %(this_position,channel))
        #find cells in this position
        is_this_position = logs_output['Position']==this_position #dtype:bool
        this_position_index = logs_output.index[is_this_position==True].tolist()#list of row#
        #
        imageshow(movie_dir,this_movie_name,logs_output,interval, this_position_index,cmap_type='gray')

In [12]:
def test():
    ROI_dir_name = 'ROI_test'
    movie_dir_name='Movie_test'#Make a path object
    channel = 'CFP'
    interval = 20 #min

    logs_output = extract_all(ROI_dir_name,0)
    plot_all_images(logs_output,movie_dir_name,channel,interval)

In [13]:
#test()

In [14]:
def makevideo_single(movie_dir,Tracking_dir,this_movie_name,logs_output,interval, this_position_index):
    """ddd"""
    import imageio #Read the multiframe.tiff movie
    import matplotlib.pyplot as plt 
    %matplotlib inline
    from matplotlib import cm
    from pathlib import Path
    #information for accessing the original movie
    vol = imageio.volread(movie_dir / this_movie_name)# define movie path 'Movie/Pos100CFP.tif'
    nslices,x_size,y_size = vol.shape #number of elements along each axis
    
    #setup for videowriter
    import numpy as np
    import matplotlib
    matplotlib.use("Agg")
    import matplotlib.pyplot as plt
    import matplotlib.animation as manimation
    #For the 1st time, install FFMPEG by uncommeting the line below
    #pip install imageio-ffmpeg
    FFMpegWriter = manimation.writers['ffmpeg']
    metadata = dict(title=this_movie_name, artist='Matplotlib',
                    comment='Movie support!')
    writer = FFMpegWriter(fps=5, metadata=metadata)

    fig = plt.figure()
    
    this_video_name = this_movie_name.replace(".tif",".mp4")       
    this_video_path = str(Tracking_dir / this_video_name)
    with writer.saving(fig, this_video_path, 300):
    #(fig,movie name, resolution)
    
        #show frame by frame
        for j in range(nslices):  
            image=vol[j]#starts from slice 0

            plt.imshow(image,cmap='gray')
            #plt.title(this_movie_name)
            plt.axis('off')
            #Labell all cell_ids for each frame
            style=dict(size=8,color='white')
            this_time = '{0:.1f}'.format(j*interval/60) #20min/frame, in the unit of hour
            time_label = ('%shr' %(this_time))
            plt.text(150,50,time_label,ha='left',**style)

            #label all cell for this frame
            num_cells_this_position = len(this_position_index)
            cellid_color=iter(cm.rainbow(np.linspace(0,1,num_cells_this_position)))
            #
            for n in range(num_cells_this_position):
                cell_id = logs_output.at[this_position_index[n],'cell_id']
                #find the index of slice j from data frame
                frames = logs_output.at[this_position_index[n],'frames']
                if (j+1 in frames)==True:
                    #this_frame_index = frames.index(j+1)#works if there is no repeat
                    this_frame_index = max(idx for idx,val in enumerate(frames) if val ==(j+1))
                    x_coor = logs_output.at[this_position_index[n],'x_coor'][this_frame_index]
                    y_coor = logs_output.at[this_position_index[n],'y_coor'][this_frame_index]
                    labelcolor = next(cellid_color)
                    plt.text(x_coor,y_coor,cell_id,ha='left',size=8,color = labelcolor)#all images were shown in the same fig
                else:
                    continue
            #plt.show() #Agg is non-GUI backend, so cannot show the figure
            writer.grab_frame()  
            fig.clear()
        fig.clear()
            
#for each slice, label filtered_cells for that position
def makevideo_all(logs_output,movie_dir_name,channel,interval):
    from pathlib import Path
    Tracking_dir = Path("Trackingvideos")#can't make a path if it exists already
    if Tracking_dir.exists():
        print("Tracking_dir exists")
    else:
        print("Make Tracking_dir")
        Tracking_dir.mkdir()
    movie_dir=Path(movie_dir_name)#Make a path object
    
    #Find all positions
    unique_position = logs_output['Position'].unique().tolist() #['Pos01', 'Pos10']
    num_position = len(unique_position)
    #for each posxxCFP.tif movie
    for i in range(num_position):
        #
        this_position = unique_position[i]  
        this_movie_name = ('%s%s.tif' %(this_position,channel))
        #find cells in this position
        is_this_position = logs_output['Position']==this_position #dtype:bool
        this_position_index = logs_output.index[is_this_position==True].tolist()#list of row#
        
        #      
        makevideo_single(movie_dir,Tracking_dir,this_movie_name,logs_output,interval, this_position_index)

# Final analysis

In [15]:
ROI_dir_name = 'ROI'
movie_dir_name='Movie'#Make a path object
channel = 'CFP'
interval = 20 #min

logs_output = extract_all(ROI_dir_name,0)
makevideo_all(logs_output,movie_dir_name,channel,interval)

Tracking_dir exists


posx and posy should be finite values
